In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import cv2   #  (OpenCV)
# OpenCV (Open Source Computer Vision Library) is a popular open-source computer vision and machine learning software library. 
# It provides various tools and functions for image and video analysis, manipulation, and computer vision tasks. 
# It is widely used in fields such as robotics, image processing, and artificial intelligence.
import os
# The os module in Python provides a way of using operating system-dependent functionality, such as reading or writing files and directories. 
# It is commonly used for tasks related to file and directory manipulation.
import PIL   #  (Python Imaging Library, often referred to as Pillow)
# The PIL library is a Python library that provides capabilities for opening, manipulating, and saving various image file formats. 
# It is widely used for basic image processing tasks, such as resizing, cropping, and applying filters to images.


In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin = dataset_url, cache_dir = '.', untar = True)


228813984/228813984 [==============================] - 39s 0us/step
